---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [67]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)


'tProblems renal cell cancer : s/p nephrectomy 1977\n'

In [77]:
def date_sorter() :
    import numpy as np
    from datetime import datetime
    
    # 04/20/2009; 04/20/09; 4/20/09; 4/3/09
    a_1 = df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{2})\b')
    a_2 = df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{4})\b')
    a = pd.concat([a_1, a_2])
    a.reset_index(inplace=True)
    a_index = a['level_0']
    
    # Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009
    b = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.]* )((?:\d{1,2}[?:, -]*)\d{4})')
    b.reset_index(inplace=True)
    b_index = b['level_0']
    
    # 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009; Feb 2009; Sep 2009; Oct 2010
    c = df.str.extractall(r'((?:\d{1,2} ))?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[?:, -]* )(\d{4})')
    c.reset_index(inplace=True)
    c_index = c['level_0']
    
    # 6/2008; 12/2009
    d = df.str.extractall(r'(\d{1,2})[/](\d{4})')
    d.reset_index(inplace=True)
    d_index = d['level_0']
    #filter lines that has already been calculated in case A
    d = d[~d['level_0'].isin(a_index.values)]
    
    # 2009; 2010
    e_1 = df.str.extractall(r'[a-z]?[^0-9](\d{4})[^0-9]')
    e_2 = df.str.extractall(r'^(\d{4})[^0-9]')
    e = pd.concat([e_1, e_2])
    e.reset_index(inplace=True)
    e_index = e['level_0']
    # Filter lines which are already calculated in
    e =e[~e['level_0'].isin(b_index.values) & ~e['level_0'].isin(c_index.values) & ~e['level_0'].isin(d_index.values)]
    
    #String conversion and completion according to the rule for 2 digits only
    a.columns=['level_0', 'match', 'month', 'day', 'year']
    a['year'] = a['year'].apply(str)
    a['year'] = a['year'].apply(lambda x: '19' + x if len(x)<=2 else x)
    a['day'] = a['day'].apply(str)
    a['day'] = a['day'].apply(lambda x: '0' + x if len(x)==1 else x)
    a['month'] = a['month'].apply(str)
    a['month'] = a['month'].apply(lambda x: '0' + x if len(x)==1 else x)
    
    b[1] = b[1].apply(lambda x: x.replace(',', ''))
    b['day'] = b[1].apply(lambda x: x.split(' ')[0])
    b['year'] = b[1].apply(lambda x: x.split(' ')[1])
    b.columns=['level_0', 'match', 'month', 'day-year', 'day', 'year']
    b.drop('day-year', axis=1, inplace=True) 
    
    c.columns=['level_0', 'match', 'day', 'month', 'year']
    c['day'] = c['day'].replace(np.nan, -1)
    c['day'] = c['day'].apply(lambda x: 1 if int(x)==-1 else x)
    c['month'] = c.month.apply(lambda x: x[:3])
    c['month'] = pd.to_datetime(c.month, format='%b').dt.month
    
    # Complete when day is missing
    d.columns=['level_0', 'match', 'month', 'year']
    d['day'] = 1
    # Complete when day and month are missing  
    e.columns=['level_0', 'match', 'year']
    e['day'] = 1
    e['month'] = 1
    
    result = pd.concat([a, b, c, d, e])
    
    result['day'] = result['day'].apply(str)
    result['day'] = result['day'].apply(lambda x: '0' + x if len(x)==1 else x)
    result['month'] = result['month'].apply(str)
    result['month'] = result['month'].apply(lambda x: '0' + x if len(x)==1 else x)
    
    result['date'] = pd.to_datetime(result['month'].apply(str) + '/' + result['day'].apply(str) + '/' + result['year'].apply(str))
    result = result.sort_values(by='date')
    result.set_index('level_0', inplace=True)
    final = pd.Series(result.index)
    
    return(final)

In [78]:
date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     415
17     335
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    243
472    208
473    139
474    320
475    383
476    480
477    286
478    244
479    431
480    279
481    198
482    381
483    463
484    366
485    439
486    255
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    427
495    231
496    141
497    186
498    161
499    413
Name: level_0, Length: 500, dtype: int64